# Basic Statistics

The contents in this notebook are referenced from https://spark.apache.org/docs/latest/ml-statistics.html.

## Correlation

Calculating the correlation between two series of data is a common operation in Statistics. In spark.ml we provide the flexibility to calculate pairwise correlations among many series. The supported correlation methods are currently Pearson’s and Spearman’s correlation.

Correlation computes the correlation matrix for the input Dataset of Vectors using the specified method. The output will be a DataFrame that contains the correlation matrix of the column of vectors.



In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("CorrelationExample")\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/06 21:31:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

data = [
    (Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),),
    (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
    (Vectors.dense([6.0, 7.0, 0.0, 8.0]),),
    (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)
]

data_df = spark.createDataFrame(data, ['features'])

data_df.show()

+--------------------+
|            features|
+--------------------+
|(4,[0,3],[1.0,-2.0])|
|   [4.0,5.0,0.0,3.0]|
|   [6.0,7.0,0.0,8.0]|
| (4,[0,3],[9.0,1.0])|
+--------------------+



In [4]:
r1 = Correlation.corr(data_df, 'features').head()
print(f"Pearson correlation matrix:\n{r1[0]}")

r2 = Correlation.corr(data_df, 'features', 'spearman').head()
print(f"Spearman correlation matrix:\n{r2[0]}")

22/01/06 21:33:52 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/01/06 21:33:52 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/01/06 21:33:52 WARN PearsonCorrelation: Pearson correlation matrix contains NaN values.
/Users/winsonyeap/miniconda3/envs/sandbox/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Pearson correlation matrix:
DenseMatrix([[1.        , 0.05564149,        nan, 0.40047142],
             [0.05564149, 1.        ,        nan, 0.91359586],
             [       nan,        nan, 1.        ,        nan],
             [0.40047142, 0.91359586,        nan, 1.        ]])
Spearman correlation matrix:
DenseMatrix([[1.        , 0.10540926,        nan, 0.4       ],
             [0.10540926, 1.        ,        nan, 0.9486833 ],
             [       nan,        nan, 1.        ,        nan],
             [0.4       , 0.9486833 ,        nan, 1.        ]])


22/01/06 21:33:53 WARN PearsonCorrelation: Pearson correlation matrix contains NaN values.


## Hypothesis Testing

Hypothesis testing is a powerful tool in statistics to determine whether a result is statistically significant, whether this result occurred by chance or not. spark.ml currently supports Pearson’s Chi-squared ( χ2) tests for independence.

### ChiSquareTest

ChiSquareTest conducts Pearson’s independence test for every feature against the label. For each feature, the (feature, label) pairs are converted into a contingency matrix for which the Chi-squared statistic is computed. All label and feature values must be categorical.

In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest

data = [(0.0, Vectors.dense(0.5, 10.0)),
        (0.0, Vectors.dense(1.5, 20.0)),
        (1.0, Vectors.dense(1.5, 30.0)),
        (0.0, Vectors.dense(3.5, 30.0)),
        (0.0, Vectors.dense(3.5, 40.0)),
        (1.0, Vectors.dense(3.5, 40.0))]
data_df = spark.createDataFrame(data, ['label', 'features'])

data_df.show()

+-----+----------+
|label|  features|
+-----+----------+
|  0.0|[0.5,10.0]|
|  0.0|[1.5,20.0]|
|  1.0|[1.5,30.0]|
|  0.0|[3.5,30.0]|
|  0.0|[3.5,40.0]|
|  1.0|[3.5,40.0]|
+-----+----------+



In [6]:
r = ChiSquareTest.test(data_df, 'features', 'label').head()
print(f"pValues: {r.pValues}")
print(f"degressOfFreedom: {r.degreesOfFreedom}")
print(f"statistics: {r.statistics}")

/Users/winsonyeap/miniconda3/envs/sandbox/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


pValues: [0.6872892787909721,0.6822703303362126]
degressOfFreedom: [2, 3]
statistics: [0.75,1.5]


## Summarizer

We provide vector column summary statistics for Dataframe through Summarizer. Available metrics are the column-wise max, min, mean, sum, variance, std, and number of nonzeros, as well as the total count.

In [8]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Summarizer

# Getting the SparkContext
sc = spark.sparkContext

data_df = sc.parallelize([Row(weight=1.0, features=Vectors.dense(1.0, 1.0, 1.0)),
                          Row(weight=0.0, features=Vectors.dense(1.0, 2.0, 3.0))]).toDF()
data_df.show()

# Create summarizer for multiple metrics "mean" and "count"
summarizer = Summarizer.metrics('mean', 'count')

# Compute statistics for multiple metrics with weight
data_df.select(summarizer.summary(data_df.features, data_df.weight)).show(truncate=False)

# Compute statistics for multiple metrics without weight
data_df.select(summarizer.summary(data_df.features)).show(truncate=False)

# Compute statistics for single metric 'mean' with weight
data_df.select(Summarizer.mean(data_df.features, data_df.weight)).show(truncate=False)

# Compute statistics for single metric 'mean' without weight
data_df.select(Summarizer.mean(data_df.features)).show(truncate=False)

+------+-------------+
|weight|     features|
+------+-------------+
|   1.0|[1.0,1.0,1.0]|
|   0.0|[1.0,2.0,3.0]|
+------+-------------+

+-----------------------------------+
|aggregate_metrics(features, weight)|
+-----------------------------------+
|{[1.0,1.0,1.0], 1}                 |
+-----------------------------------+

+--------------------------------+
|aggregate_metrics(features, 1.0)|
+--------------------------------+
|{[1.0,1.5,2.0], 2}              |
+--------------------------------+

+--------------+
|mean(features)|
+--------------+
|[1.0,1.0,1.0] |
+--------------+

+--------------+
|mean(features)|
+--------------+
|[1.0,1.5,2.0] |
+--------------+



# Close all sessions

In [10]:
spark.stop()